In [ ]:
data_dir = Path("data")  # folder where you've saved the exports

files = {
    "UB":  data_dir / "ub_ec3_properties.xlsx"
    # add others here...
}

frames = []

for family, fp in files.items():
    # Inspect header rows in Excel the first time to set header row correctly
    df_raw = pd.read_excel(fp)
    
    # --- 1. Clean & standardise column names ---
    df = df_raw.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace("[()]", "", regex=True)
    )
    
    # --- 2. Add meta columns ---
    df["family"] = family   # UB / UC / PFC etc.
    
    # Often Blue Book has a 'Section designation' column. Standardise the name:
    if "section_designation" in df.columns:
        df.rename(columns={"section_designation": "designation"}, inplace=True)
    
    frames.append(df)

bluebook_df = pd.concat(frames, ignore_index=True)
bluebook_df["designation"] = bluebook_df["designation"].ffill()
bluebook_df["designation_mass"] = bluebook_df["designation_mass"].str.replace(" ", "", regex=False)
bluebook_df["designation"] = bluebook_df["designation"] + bluebook_df["designation_mass"]
bluebook_df = bluebook_df.drop(columns=["designation_mass"])
bluebook_df["designation"] = bluebook_df["designation"].str.replace(" ", "", regex=False)

# Optional: set index for quick look-up
bluebook_df.set_index(["family", "designation"], inplace=True)